In [1]:
from transformers import BertModel
from keybert import KeyBERT
from kiwipiepy import Kiwi
from tqdm import tqdm
import pandas as pd
import numpy as np
import xmltodict
import requests
import json
import os

os.getcwd()

In [2]:
with open('description_lst.json', 'r', encoding='utf8') as f:
    description_lst = json.load(f)

In [3]:
df = pd.DataFrame(description_lst, columns=['idx', 'dscription'])
filtered_df = df[~((df['dscription'].isna()) | (df['dscription'] == 'ISBN'))].copy()

In [ ]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

In [ ]:
kiwi = Kiwi()

keyword_lst = []
for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
    result_text = ''
    index, description = row.tolist()
    for sentence in kiwi.analyze(description):
        nouns = []
        for token in sentence[0]:
            if token.tag.startswith('NN'):
                nouns.append(token.form)
        if nouns:
            result_text = ' '.join(nouns)

    keyword = kw_model.extract_keywords(result_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
    keyword = [k[0] for k in keyword]
    keyword_lst.append(keyword)

In [ ]:
filtered_df['keyword'] = keyword_lst

In [ ]:
filtered_df.head()